# In-Depth: Support Vector Machines

Support vector machines (SVMs) are a particularly powerful and flexible class of supervised algorithms for both classification and regression. In this section, we will develop the intuition behind support vector machines and their use in classification problems. We begin with the standard imports:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
# use Seaborn plotting defaults
import seaborn as sns; sns.set()

# Motivating Support Vector Machines

As part of our discussion of Bayesian classification (see “In Depth: Naive Bayes Classification” on page 382), we learned a simple model describing the distribution of each underlying class, and used these generative models to probabilistically determine labels for new points. That was an example of generative classification; here we will consider instead discriminative classification: rather than modeling each class, we simply find a line or curve (in two dimensions) or manifold (in multiple dimensions) that divides the classes from each other.

As an example of this, consider the simple case of a classification task, in which the two classes of points are well separated (Figure 5-53):

In [ ]:
from sklearn.datasets.samples_generator import make_blobs
X, y = make_blobs(n_samples=50, centers=2,
                  random_state=0, cluster_std=0.60)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

> **Figure 5-53: _Simple data for classification_**<br><img width="300" src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAeYAAAFVCAIAAAC1r9tDAAAACXBIWXMAAAsSAAALEgHS3X78AAAgAElEQVR42u3dZ1wU194H8DNb6UhHLFQLIkqxISCggiAWRFFp1ng11RtNsSRqmvqYxJubRHPtBXvvjSZWpKgIqFjAhoo0gYVdts3zYpXggl7wLuws+/t+8iJ7GNkz/zn8dvZMo2iaJgAAoAlYKAEAACIbAAAQ2QAAiGwAAEBkAwAAIhsAQMtwmr7o2rVrk5KSJBJJVFTU2LFjUTsAAIZGdlpa2rVr13bt2lVTU7Nx40YUDgCg9VFNvJRm5cqVFEXdvXu3urr6q6++cnFxQe0AABi6l11eXv706dM1a9Y8fvz4ww8/PHXqFGoHAMDQyG7Xrp2joyOHw7G3t+fz+WVlZaampg0Xo2maoiiUFQBAnZHt6ekZFxc3ZcqUoqIikUhkYmLS6GIURRUXVzFqDS0sDJnWJfQKhUKh0CWlXqk4sv39/TMyMsaNG0fT9OLFi7ErDQDA3L1sQsgXX3yBegEAqBEupQEAQGQDAAAiGwAAkQ0AAIhsAABAZAMAILIBAACRDQAAiGwAAEQ2AAAgsgEAAJENAIDIBgAARDYAACCyAQDaJA5K0KJomr60f4/ozCm2sEbUw2XgR58ZGbdDWQAAkc1ExxZ8OXzzBmuZjBAiO31ye2JCv+17zaysUBkAeA+YGGlBd7Ou9dm5XZHXhBA2IbE3rqf99jMqAwCIbMZ5cOxor5rq+i0UIXrXrqIyAIDIZh42i27QJmezURgAQGQzTveIiWnGxvVbZISI+vZHZQAAkc04nRyd7v/jo1xdPcXLakI2ePv6fjEPlQGA94MzRlpW4Jfzc7x9bxw6wBLWUO6eoTGTeTweygIAiGyG6jnQp+dAH9QBAP53mBgBAEBkAwAAIhsAAJENAMxVU1MjFApRB8DhR9AYtzOuPNq1g1tRUevoNGDmR+1MTLVhre9lpuev/Nn0eiZNWOV9+nT5cr59z14YDIhsAEY7v36N/bLvo6uqCCEyQvaeONZ98/YODo5te62LnjwunfVB9MOCV69PHt9/547x0dOm5uYYEtoJEyOgAaqqKvmrf3evqlK8ZBMy8fbNnJ+XtvkVv7ZmdWhdXhNCCBlz/27a2tUYEohsAOZK27836MljpUadjHSaptv2ius8fkQ1+IvlP3yAIYHIBmAuWi5v2EhpwYqLG5uvF5uaYUggsgGYq9+48fEdOio1Cj08KaqN53bnqJirb95Z7KKZeZdJUzEkENkAzGVkZFwz6+NsA0PFSzkhe7t27zFXI2+wJZFIJBJJExfu3rf/40U/HHB0qiKkgpB9Xbu//H6Zg3MPDAmthTNGQDP4zfw4t1fvHXt38SoqhfYO/WZ9onFnTeRnXbu/8mfDaxk0xarw7OP81ULb7s7/9V8NjJ1SOz4y4eRxis32GhaC24ohspshPDzcwMCAENKxY8elS5eifNCaXLx8XLw09QZbRYVPimZOjcrPf/X62JE9d+4YHz3VlLPL+Xy+T1g4BgA0L7LFYjEhZOvWragaQHNdXbs6ti6vCSGEjL1ze+ea1cPmfYPiQNM1Yy779u3bNTU106dPnzJlSlZWFmoH0HR6Dx8oHSplE8J/85xrAFXuZevo6EyfPj0iIuLBgwczZsw4ffo0i4WjlwBNUtvOpImNAO9ANf1iBLFYTNM0n88nhERERPz5559WVlaoIEBTZJ0/zxoxwrWysq4lzczMJCmpSy/cMARaZi97//79d+7cWbx4cVFRUXV1tYWFRaOLFRdXMWoNLSwMmdYl9EoLC2XT3e3CwsX5q/8c9rBARshpRyfe7C/s29urau0wojS9UKqP7HHjxs2fPz8qKorFYi1duhSzIgDN4jN1Rs2E6CNHDrLY7P4jw3R0dFATaMG9bC6X+8svv6BkAO9NT08vYGI06gDvDXvKAACIbAAAUDVcsK5hpFJp0m+/cM+nsEVCYQ8Xl49md+zSFWUBQGQD49A0fXjmtClHD706bnXt6uHUy2TrLqQ2gJbAxIgmyTqbFHz6RP3zDEbfv5f71x+oDAD2sjWMRCK5uH2r/PpVqa6e3fiJXd09297WKrmQEigWKzXq3szFOAZAZGsSgUBwJnZC1MXzivPRr+3ekTzny4BP/tnGtpZcR5du8DQWqa4uxjGAlmgjEyPnVyyd8TqvCSHugiqrP//9vMHTAjWdW+yURAvL+i1CQmSD/DGOARDZmkQ3LVVpTQaVlWbv2t7Gtpaldfvqr7+Jt7RUPAkxn8fbGjY24NPPMY4BtEQbmRihZDLlFkKIVNr2NtjASVOKhwXvittMCYWW/oPDff0wiAEQ2RpG5OZOsq7Vb7luYOg4MqxNbjMLK+vAL+Zh7AJooTYyMeI2e+7OHj3rbiNbxGZfnRDp4NITG5j5Kisrsi5dKHr2FKUA0Ja9bKuOnbh7D29b/YfO7VyZnp5OUPCI8VHYuo0qLS7O2PAfbvELWcfOAz6YaWhopK6eyGSyU9/Oa3/sSK/nz/KNjdMG+fv9+rvRO+/6X/GyPH3/XhaH02/seMVjSAEQ2RrJ1MIiePH32KLvlpOSXD3ns6jHD1mESAg5uH+P3X822PdUz13245d+H7F+jT4hhJD2FRUDjx7eKJGO2rrzbcufXf2H4ZpVEc+eygk58/tK2ew5AydNa8obPX74QFBR0aWHC4eDy31Bs+HqRy1C0/Sz5T+OePxQsdW5hIy/k3dnqXo+56RSqf6p4/r1WihC3C+kFNy+1ejyN1KSXVYsDX72lEMIj5ARjx/Z/Ljk7psHMBoqyLlxatxo4tuvw1Dfi0N9L23ZgGEAiGzQDDevZva/flWpsXP6ldLS0tbvTFVVpfmLIqXGHgLBo+zGnwT9fN/uHjXV9Vv6vnxZsPNd53GKRKJ7H8+MPZfsIRI5EjLuZm7XJd9ePXEMIwEQ2aABJKIa3QZnQ/LFEolE3PqdMTIyLrLpoNSYZWzs4Nm30eV5FRWNNFZWvOMtLsdtGnPrjav5nasFpXt2YiQAIhs0gEs/r4vdnZUa83u7WVlZt35n2Gy2ZGRYGfX35fdSQrL9h3ZycGx0eaGjk9KDpSWEiB2d3vEW8sePGl7Lzy96jpEAmgtHY7QIl8vl/eOjG0sW9nr9nO+LFpbmH35KUZRq3+hu1rUHhw9QcrlNyIge/b3ettjQuV+flEj0jxx0elDwxMLyacCQwGVvfVJdn1kfHzx9MvzenbqWvS49vf7x4bv2Rzrb1RCi92ajqH17jATQXBRN06r9jXigMsN7lX0u5dmu7bziInGHjg6TpznVu+WhSnp18ofFvTeu610tIITk6ehcmBA9csXKd3wq1NbWPn1aaG5u3ujphvW79PD2rbyVK3SuZhAWS+jZr9fXC9vb2b2jJyKRKGX4kJic7LqWXEPDp6vXuw0L+a9rQdN05pnTL+NPUnK5nl9A/5Fh9Z9PjQeHa26vNP0J64hs9EqVvbqenGg/aaJTbW1dSxGLdfmPNd4RE1TVJZqmm/614OHtW7lLFna7ctm4puZGT1fdaf/wip7UlLw+8sU/g3bGdZZKFatwcNSYsL/Ws9lsJBEiW72RjYkRUOkH9tHDgfXymhBiJZeLEk6R943sRvYymjONY9vd2XbXgWfPnr4UCPwcHOsy990yTh4P3rG14+tDtVZyedSh/acH+vhPmY5NDOqFw4+g0vH0Zl4rsEW16u1V+/Y2Tl26NjGvCSEV8ac6vnlqjREhsvNnsX0BkQ1tCuXuUf1mi4QQsWsvDfuraHAqJCGEJZNj+wIiG9oUn0lTt/n61d30liYkrk/fgbM+0ay14Hp5l73ZUkuI7C0njAO0JsxlgyrxeLzguN07f/tFJy2VyOVC9z5+n8/VuPs3DRwfuefUiUknjylOEBQTssXXL2TGLGxfQGRDW6Onpxe8YJFGrwKbzR69YevRDWuoSxeIXC7t0y9k5sc6OjpKi4lEoss7t0lLS9oP9Ok50AebHhDZoL1qa2sv74jjlxVJrDp5TYjkcrmt+ofB4Qye+TGZ+fHbFsg9d/bF/C9G3r2jS8hdPv/AkKDQNRv5fL7KeyKTyVgslsovdwJENjRJfk72nfX/MX78oEbPgD98pE9kDGrS0IPc7DufzAzLzdEnpJKQA1s3uq/ZaGPvwJDuicXi59/Mm3j31aWYXWpr7U4c3bn0u5DvlqrwXdL37anYtlnv/r1aExNRwJAhC5fweDyMDUQ2tJ47aanCWdNjXz/6/Vli/Kl7d4O//Q6VUXJ70YLY3BzF/xsRMuX61S2LFtjE7WJI91J27Bhx+2b9Fi4huufOqvAtMvbvsfvynz2qBYQQUvS89vatuBcvRv+1HmNDy+GMkdbdeVz1+9DXeU0IaS+Vdtq5rbit3KiIpumrCWcSl/+YtPav6urq9/49Bffu9kq/otRon3qppKSEIWsqLC5ueESVXaXKa+pebtvyKq8JIYTwCfE4fSL/1k38ESGyofXo5txQavErKc46dKANrFpNTc3B6PHOkyMnrlwR9s3XGUN9cpIT3+9XCcrKTEUipcZ21QKBgCnXGbsMH55pqHyFsbBbdxV++Onm31Nq9BAI8lW6Iw+IbPgv5A0OT4kI4Rq2hWcYpvyw+IOE050lEsUu4bj7918sWSiRSN7jV3Vzc0936qrUeLOHS+fOtgxZWUcXl2sjRtf/HnHZ3KLDjA9V9fspipK0M1VqLGKxDBlTAUBka4Xq/l5Kd+E64ejUP3x8W/gCcfGc0vXgobduph7c/x6/isfjySdNydf5+2bXNw0M9abOqH8vPbUbsfKPw18v3NOn34Gu3baHjKj9zwYX/8Eq/P2iIYFKXzROeXj2bcI9CKFtw+HHVuW96Pt19+9FpF4yIUROyBnr9gbzFzU84Zf5LmzdLDp6iFdSLO5saxEzuXfgMI5AoLSMPiGi0uL3+/1+sz5Js7S+sn+PflmxwMLaPDLGKySUURVgs9mBc78mc79uod8/ZP63cUXP3U6f6FtZ+ZzFOuXh6fLzb4z60AK1wM1XW5tUKr20Zyev4E4lV9djygfmlpYaV6v4FUv9//2rzetJjyxj46f/t7J8/56o+NP1F0s1bsc/c7bj/3ZmnpbfU/Thnbw755LbdbbrEzjs3adm4+armj6iWmQvu7S0dOzYsZs2bbK3t8fH3Xt+r+FwBkXFMnPcNIVAUNVu1zabepPUvSsqcjeubf/FvEvXrw0sfqFofEnIzbCxIxlzJrWGsu3azbZrN9QB3ieypVLp4sWLNfFbPKhQVnLi4CdPlBo73bpp4er2Yt3mbRvW6ubflxgZkcDg0I8+RbkA1BbZ//d//xcZGblmzRpUTZsZW1mVcjhGUmn9xip9g866us4DfZxxqw2AltTUoxkHDhwwMzPz9vZW+dw3aBaXvgMuePSp30IT8sLLW09PD8UBaGlNPfwYExOjOPpx+/Zte3v7v/76y8zMDOXTTnlpaTc++CA0O1uPkBKKOu7rG7pnj7mVFSoDwJTIrhMbG/v999+/4/AjDhBrQ68Ut9mTPi3U7+HSf3R4y518hs2HQmlJoZq4ZLPPy8ZNIIEQwufz/ad+oFWrLJFIkv/1M/fcWbawhri7dZ46087FFSMBWlmzI3vr1q2oGmgbmqaPzJo25ejhV+dLZd84kXKebNlh18MFxYHWhIupAP67G+fOBp0+Wf/81uEPC/L++gOVAabvZQMwcy846c/fyKkT/LJSka2dafRkj5GjVfj7i8+nDBWLlRp1b+NWqIDIBmi+k9/OH7XuLxPFsfT793LSr6TX1vYdp7L7bdEGBjJClO57JdXTR+WhlWFiBDReyYsXnQ7uNal37lPPqqqKLRtUeA2BZ8yUM9bt67fUECLzC0DxAZEN0Dw3Ek77FivfMtD87p3/5ck4SkzNzeXfLDnR3kZGCCHkro5O3JhxAZ/NYXhlRCJRwp+/JU+NSZw59fyu7bgOrg3AxAhoPNOOnZ6z2Z1ksvqNAkND1d4Pp9/4yPKhQbu3baWqBc5jRoZ3d2N4WWpqak5HR0y+eF7xWI2SQwf2Xzw/5ve/cJ4u9rIB1Km3r1/ym9fQywl56TOIw1HxHomJqVngZ58Pnf+tm68v88tyYdW/p73Oa0KIOU2H7N9zNeEMBgwiG0CdKIrqtuznuF69ywkhhOTzeOuGBvn/sFzLy8LNTFf6yOoslZYnnMaA0WiYGIG2wKGXW+dTyWcP7hMVPrH06BM+yB81IY1NgNB4rg0iG7SQXC5PP32y6tFDB59BDi49GTGUORyfiInYNHWk/b3EifG8ei33eTzL4FBURqPhIxea7UFu9qkRgX2nRkd+O483ctihmdPEDS4zAbUb9OGnGwYH1t0A6QmbfTYqthe+f2AvG7QKTdM3v/p8cka64qW7oMrl4L7dVlbB3y9DcRiFz+ePjtt1ekec7MplOZdrPGz4yJBQnC6CyAbtkpFwJigzo34LjxCdpET6OxpxwDRcLtdv8jQyeRpK0WZgYgSap/xhgbVcrrxDV/FS3qARABDZoGbd/YekGxsrNVY7OLLZbBQHAJEN70kmkwkEApVfo9zZqcuN4FBhvZZb+gbGMVNQcIBWgLnsNkgkEiUuXmCUkmxUXl7i4KgbGTtwkiojNfRff+6ztOYnnuGVlwkdHI2jJ6vwnnnATHK5/OHDB3p6+lZ4yCciG1Tr1Cezphw5wFW8yCzLz825xKIGxkxW2aDhcIK/XUK+XULTOOSoFa7s2l6zYa1zzo0KXd2r/b16LP7RtrszyqIWmBhpa/Jzc/omnuHWa3EQCat372iJ90Jea4OsxDOdv/l6fNY1V5nMWyCYlBh/66MPamtrURlENqjA/UsX3KoFSo36DwpwRge8nxc7tvWqrKzfMjYn+1LcZlQGkQ0qYOrg9KTBHexqzcxZuLkEvBf+s6dKLXqEyB8WoDKIbFABj8FD4j371m+pIaQ2MAiVgfcjtrJWahERQnXoiMogskEFKIpy+9efm7x97/N4UkLOm5rtipkcOO9bVAbe83tbxMTb+m885fJQ9x5ek6ejMmqBM0baoI5OXTocOJZz5XL6vbsu/oNHduyEmsD7f28bPuLioh+yN63rd/vWSz7/Rt/+XZf8qKuri8ogskGV+9quAwaSAQNRCvjfeU/9QBIz+faN6wbG7YKduqAgiGwAYDQul+v65jESUAvMZQMAILIBAACRDQCAyAYAAKbD4UdtJBAI0o8d5unr9w8ZweFgDAAgsoGpklf9rr9x7ejHj4SEnHHu0e6rBe6ho1AWAI2AiRHtcvXUcbcVS0MfP+IT0o6Q8bdu0vO/KHpaiMoAILKBccoO7usqrKnfEvT8+fVN61EZAEQ2MA6nrEyphSKEW1aKygAgsoFxajvbKbWICCG4BBmg7UW2XC5fsGBBZGRkdHT0vXv3UDtN1HX6PxJtOtRv2dOzl9eUD1AZgLYW2UlJSRRF7dy5c/bs2StXrkTtNJF9DxfuqnXbhwSesLA8YmOzdXS428ZtuCsbgKZoxkl+Q4cOHTx4MCGksLDQ2NgYtdNQzt4+zt4+IpGIzWZzuVwUBECDUDRNN+sfzJs3LyEh4ffffx84EDf2BABgdmQTQkpLSyMiIk6cOKGjo9Pwp8XFVYxaQwsLQ6Z1Cb1CoVAodEmpV01cshlz2YcPH167di0hhM/ns1gsPP4VAKCVNWMuOygoaP78+TExMVKpdOHChTweD+UDAGBoZOvq6v72228oGQCAumByAwAAkQ0AAIhsAABENgAAILIBAACRDQCAyAYAAEQ2AAAgsgEAENkAAIDIBgAARDYAACIbAAAQ2QAAgMgGAEBkAwAAIhsAABDZAACIbAAAQGQDAAAiGwAAkQ0AAIhsAABAZAMAILIBAACRDQAAiGwAAEQ2AAAgsgEAENkoAQAAIhsAABDZAACIbAAAQGQDAIBqcFACAGh75HJ5cvK/WKwTHE5pba2Dmdlkd/fRiGwAACY6ceLL8PB1xsaKV/nZ2WkZGeI+fSI0fb2aOjEilUq/+uqr6Ojo8ePHJyUlYUAAMJlMJhMIqmia1s7VLyoqdHA48DqvCSHE1bWyvHyjFu1lHzlyxMTEZMWKFRUVFWFhYYMHD8ZfBQADSaXS+Pgl+vqnjI1LSks7s1jj/f0/0bYi5OScHj++VKnRxORObW0tIYZaEdkhISHBwcGEELlczuFgOgWAoU6e/DwycouuruJVWWFhbnKyPCDgM60qgrFxp+fPWTY28vqNNTVGXC5XW/aydXV1CSECgWD27Nmff/75O5a0sGDchxgDu4ReoVAt0aWioqddux5/ndeEENKhg4TL3W9mNp/FYmlPoYKDx+ze3W/ixNS6FpmMyGRDrayMGTuimohq+mzXs2fPPvnkk5iYmDFjxrxjseLiKqaNY6Z1Cb1CoVqoSxcu7AsKmqav/8aPLlwwNDe/YWZmplWFys+/dufOZyNGZJmYkHv3+ElJg4OCNurr6zN2RKl4L7ukpGT69OmLFi0aMGAAvnsCMJO1dbf8fF1XV+Gbf7wW9vaG2lYKBwf3zp2Tz57dJxQ+sbbuN2bMoLaxXk2N7DVr1lRWVq5evXrVqlUURa1fv57H4+EvBIBRnJxcjx71dnVNqGsRi0l5+WDt/GvlcDg+PhPb2ko1cbmFCxcuXLgQfxIADDdgwJ8bNnzs7X2xSxdRRoZxdnZQcPAylEXrIhsANIKFhc2oUQdv3cq4di2nWzef0aOdUBNENgAwmrNzH2fnPqhD24PbQgEAILIBAACRDQCAyAYAAEQ2AAAgsgHamJqamoyM+Lt3s1EKQGQDMFpS0q9ZWQM9Pce2axdw8mTogwcIbmgEzssGUL9Ll+IGDlzWqZOYEGJpKe7R43xc3Cwbm2TcFgKwlw3AODU1BxR5XScsLPvy5R2oDCCyARiHz3+h1GJoSCSSx6gMKMHECMAbRCJRSspKLvcyRclFIndv7y+MjNq1/Jt2IOSNyeuSEkpXtxs2ByCyAd5KKpUePx45fXqi4oFTcvn5zZtTAwIOGRgYtOj7mpvH5uZedHF5det9miaHD/cLDR2LLQKIbNBqd+9ef/BgN5st4vEGeHmNY7PZ9X968WJcdHRi3QMCWSwyaVLa7t1/BgXNa9FeubuPTE19mZOzwdHxZnm5QWGh94ABy5X6pkI0TZ87t0kqjWezRUJhj/79/2lqaoGxgcgGYJbExF+7dv01KkpACCkp2bh3776RI7fx+fx6e9lphm8+v4XDIVzutVbo24ABsTQd8+zZUwcHAzc34xZ9ryNHPgsP32pqShNCaDpxx46z7u57LSxsMEIQ2QBM8fDhHTu73zw8BK/nIujp00/v3furtXX/oqIdXG6xWNyxpKSy4T+Uy3Vap4cURdnYdGjpd7l5M9XHZ48irwkhFEWiorK3b185bNgvGCSIbACmyMvbFRVVUb+FxyM1NXutrFYNGfJqEjklxeTIEc6oUdK6ZUpKKC53aFuqQ2HhST8/4ZsfFURH5xpGiEbASX6gNWOdJaaoho2FPXv+/bxtP7/yx4+tbtzQU7y8f5938GCsr29MW6oDTXNoumEjFyMEe9kADGJpGXTv3honp9p6OUW4XJHSYm5uFY8ebcvNPUuIzNIyJCxsUBurQ48e0Zcvrxs48GVdi1RKamsHYoQgsgEYpFevQQcOjDU339Gu3au8Xr/ewcMjX2kxkYjbvXsfY+OhbbUOHTs6JCX989q1X93dqwghZWVkz54hw4d/iRGCyAZglrCw1fHxnnJ5IoslFApdfX0/u3p1gqdnZv1lnj0b0KtXu7Zdh8GD59y9G7B9+y4OR8Tj9QsLm1h3QqFMJrt0aadYnEIIxeP5eXtHsliYPkVkA6gDi8Xy959ByIy6Fhubb48d+2z48EcsFhGLyYEDPbp1W6wNpejSxb1LF3elRplMdvDg1AkTDpmaEkJIaemuPXsSwsM3ILUR2QCM4OIyuLQ0eefONRzOC7ncfsCAGS19oSOTXby4LSrqkPHrk8LNzMiECfvPnh3Sxg7AIrIBNJiZmUVQ0DeoAyFEKk0xfvMiHlNTIpGkEILIZsw3RZQAAF6jm9wIiGwAUCs227eq6o2WigrC4QxCZRDZAO9JIKhKTT2Rl4er9VTP23vStm2hdaldWUm2bx/l7R2NyjAH5rJBc76003R8/I8mJjt9fJ4UFfFOnuzv7LzCzs4FlakjEolSU/dJpUJPz3ATE7NmxwGHExYWd/r0VpnsHE1THM6g8PBJLXdDQUBkQ1t2/vy6IUP+ZW0tJYSYmIi7dz+/detHHTsmcjgYxoQQkpm5v7r6p9DQe3w+SUr6OSNjWmDg3/eMvXv3WkHBdi63XCx2HDDgQ2Njk7eltp/fNEKmoZ6IbID/iURyWJHXdUaOvHb27D4fn4koztOnjziceWPHFileBgY+f/jw17S0rv36hRNCLl3abGOzKDr6JSFEKiX79h3p1m1bx45OqJvGwVw2aAweT/kBiSYmRCh8gMoQQq5c+c+QIUX1W2xtaysrDxJCXr4sJ+TXvn1fvt6PJhMn3szJWYqiIbIBWpBI1EmppbCQbWLSC5UhhHA45Q3vUygS3Y2PD92/3yU4+KHSj/T102kaZ+8hsgFajIlJzK1bf1+aSNPkxAlvT88QVIYQwmb3qK5WbqytvRcVdb5XL0HDC84pSo7IRmQDtKA+fcLv3l2+a5dnaqr+qVPWmzeP8/PbTDXct9RKQ4b8Y/fuPvVD+PBhnWHDxIQQX1+SlKS8fHW1B+4copFfp5q1dFZW1i+//BIXF4fCgVp4eU2i6dgXL15YWel7ehqgIHX4fP6AAdvj4pbo6aVyOOKaGveXL6+PHv2EEGJoSCiK3LxJevR49e3k0KEuXbt+jaK18chev3794cOH9fX1UTVQI4qirKysUIeGLCzah4SsoWmapmkWi5WYGEjIE8WPgoLI9etk925SXt5bX9/f0/MjC4v2LdeTsrKSzMwtFFXj4jKqffve2DQq1IxvRra2tqtWrULJABj+kbnjfdIAABBjSURBVKaY8aDpkJd/P3mGuLkRmcx11Kik4OAfWjSvU1PjCgoGTpz4XWTkz7q6gQcPfiCTybBd1LCXHRgYWFhYiJKBlrt1KzU19VJ1Na9//8kGBoYt90aFhQW5uZs5nGoez9PLa3xzr0IcPPifBw8+dXDY7+tbWlzMOnPG3clpBZfbss94LC5+zud/HxT06nTDLl1ENjZ7jhzpMXToHIwc1aCb48mTJxMmTKABtJJEItmyJer2bV2aJmIxOXbM/vLlvS30XomJa86fN5fLCU2T0lKyceNQgUDwHr/n6dNHR4+uvnDhqEwma4USHTr0nUxGaPqN/w4eDMDgURXOe0T8f/uYrWLYBJ8h07qEXmlol86cWR4RsUNXlxBCuFwSGlpw5Mjc/HwvQ0Mj1b5ReXmpWLx48OASxUtTUzJpUsL27V+HhCxrbqE4nHb9+8cQQkpLq1uhRDU1ZQ3PQ5HJqpizERn7p6f6uey6mTJ8NQHtxOUmKfK6TkjIo7S0zSp/o4yMrYGBz+u3sNlER+cS80tkYuLz6JHy3ItQ6IzBoyrNi+wOHTrs2rULVQPtxGZXNwhxQtOVLfBW4oYT1yyWmPkl8vAIOnEiuLb275ZjxxydnT/F4FEV3BYKoKmEwu6EZNdvuXePb2Xlp/I3cnQcefXqvz08BPUbRSINuDSfoqiRIzft3buCxzvHZgsJ8bC1ndW5c3cMHkQ2QGtzcvooMTF1yJDHrzOUJCYODw/3VfkbOTj0OHp0oq3tRjMzuaLl8GEnZ+fPNaJKPB5v2LBXz9JU1cRxZWVFevo+NpvTv/94XaXJKUQ2ALxl59fz3r3N27atNjLKE4l0xWL/kSPntdB7jRjx69mzPaXS02x2lVDo3KvXxzY2DtpZ9uTkfxsZ/TVu3FOplJw8+S8u96v+/aO0dhBSKr81DE45QK9QKHRJVb26du2Mre3kbt3+Popw4YIZn3/Szq57GytUE5fEfWEAgLlKSvbVz2tCiI9P6Z07m7W2IIhsAGAuLresscaXWlsQzGUDtAiRSHT58iaavlFSUi0W86ytLa2tQ3v29EZlmllGR5om9a8GEYuJVNoFkQ0AKlNRUZaSMiEq6ori7IasLPLsGenQYf2RI9EjR65U7/VolZUVV66sZ7PLeDxXL68Ihj9A3c3to6NHT48alV/Xsnu3q4/PTEQ2AKjMxYs/Tpt2pS6Ze/cmZWXExEQUGrrp0iUvb+/x6upYdvaZysovx48v4HBISQk5cGBLQMD2du1MGVtJa2vbmprN27b9oq9/VS5nV1f3c3dfZGCgvbdKR2QDqJ6eXprSnrSfHzl2jIwaJRcKTxOinsiWSCQvXiyaOLFA8dLcnMyYcXHr1m+GD1/N5GI6OLg5OGyTy+UUReGGGTj8CKB6FCVv0EIU59NyOLXq6tWVK4eHD7+p1Ct9/Ysa8RBIFouFvEZkA7SImho3pZb0dOLuTsRiIpG4qatXQmF5wxkFLlckl8uxyRDZAG1WWVnxyZNfnT0bmJw87NSpxdUNnm3u5vbV7t0udTuvhYXk8WPSsSOJixvg7f2Rurrt7j46OVn5GWzV1a4MPwIJ9WEuG6B5qqoqUlPDY2PTFV/TZbLLGzZkhoYe4PF4dcu0b2/H4x2Oi/uTx8t5+vSJREIcHMy3b+8TEDBXT09PXT03N7fMyIh+/vx3a2upouXCBWsbm4+wTRHZAG3WpUt/REam102rstkkOvrc8eNbAgJm1F/MzMwyJOR7pnV+2LDFFy7YicVHebxyodDBzm6Gi0v/ty1M07RAUKWjo9vSjx8DRDZAS+HxspUmEvT1CU1nEjKD+Z2nKMrXdwohU/7rkmlpuyorN1pY5FVXG5WVDfL3X96iD7oERDZAi5DJGrn5p1yu05bWMTPzoJ3dXBcXxe2TyuXyuHXrisLD92Hrqx0OPwI0j45O8IsXb+xm5+XpWlmFt6V1LC/f+jqvCSGExSKBgWdv3DiLrY/IBtAw3t4TEhNn5ua+OoqYnm6YkfGpq+ugtrSOfP4DpRYHB3FRUQa2vtphYgSgeSiKioxcdeHCuB07jhLCdnKKCApqa4+jFYvNCLlfv6WsjOjqdsLWR2QDaKRu3fp069an7a5faHl5uonJ31dFHjvmFhQ0DtsdkQ0AjDN48D+PHCnq2HG/r2/Rs2ecs2f7ODv/jCtuENkAwEQURQ0fvry0dO6hQ6fNzDqHhPji/h6IbABgNDMzi4CAGNSBUXDGCAAAIhsAABDZAACIbAAAQGQDAICK4IwRAGAogUBw+fIaDqdALDZ3c5thZdUBNUFkAwATPX6cl5c3Zdy4XB6P0DQ5c2bXs2e/urmFanlZMDECAEyUk/NdZGSu4lE/FEWGDXtaVrZUJpNhLxsAgFmqq6stLdOUGv39szMzE/v2DWrNnkil0kuXtkulV+RynqVlWK9e/ohsAIA3yOUyDkei1KijQ8TimtbshkgkOn48KioqwciIEELu3t1+8uSH6n0+HCZGAIBxDA2NiorclBqTkx09PYNbsxvnzv1r+vRXeU0I6dKl1t19bV6eOu8bjsgGACbq0OHz5GSbupc3bxrI5TN1dFr1gW0czmWlJxW7utY8enRQjWVp6sQITdNLlizJy8vj8Xg//fRTp0642TkAtCAXF/+Cgn3btq3T0XlUW2tmZTXR13doa+/SsuSNNUs1ILITEhLEYvGuXbuysrKWLVu2evVqDCkALVFYWJCTs5bHeyoW2/Ts+Y8OHexb533t7Xva2/9bjSsuFHrI5edY9SYj8vN5lpbBauxSUyM7MzPT19eXENK7d++cnBwMYgAtkZ2dIJN9Eh39lKIITZOEhINlZX+6ug7VhnX38Zm7efOlSZPSOBxCCCktpRISxoeF+WtAZAsEAkNDw1f/hsORy+UsFubBAdo4mqafPVsRHf1U8ZKiSGDg0x07VvTsOUQbHnpgaGgcEHBw9+4/ebzrUimPxwsMC4tV74o3NbINDAyqq6sV///uvLawMGRa3RnYJfQKhdKILt27d8fNTfkECVfXjKqqIkfHLtpQKAsLQ3v7pczpfFMj28PDIzk5OTg4+Pr16127dn3HksXFVUzbPEzrEnqFQmlKl0pLBQYGyo0URUpKBEZGVShU63+KNDWyAwMDL168OHHiRELIsmXL8IURQOH+/Rv372/h8cpEItu+fT82M7NoS2tnb++UkODp4nKlfmN2tueQIY7Y9Izey6Yo6rvvvkO9AOq7cmWHmdmC6OgyQohcTg4dOtKx4yZ7+95tZgUpirK0nJuc/FlAwHNFS3KytYXFXDy9l+mRDQBKxGKxWLzSy6tM8ZLFIuHh97ZtW25vv7MtrWbv3sEPHx7dtm09n/+strZ99+4f2Np2w9ZHZANomPT040OH3lFqNDXNEAqFurq6bWlNbW272dr+jC3OBDhRD+A90TTdcHqAoghN0ygOILIBmKVv39CEBOUT3crKPPT09FAcQGQDMAufz2exPsvIaPd6p5scPuzg6Pg1KgMtB3PZAO9v4MDJeXk9tm/fxuOVCYW2Hh4f4vmEgMgGYK5u3fp269YXdYDWgYkRAABENgAAILIBABDZAACAyAYAAEQ2AAAiGwAAENkAAIDIBgBAZAMAACIbAAAQ2QAAiGwAAEBkAwAAIhsAAJENAACIbAAAQGQDACCyAQAAkQ0AAIhsAABENgAAILIBAACRDQCAyAYAAEQ2AAAgsgEAENkAAIDIBgBAZDdr6fj4+Llz56JqAABqwWn6oj/99NPFixednZ1RNQAApu9le3h4LFmyBCUDAGDcXva+ffu2bNlS93LZsmUhISFpaWkoGQCAulA0TTd96bS0tN27d//6668oHABA68MZIwAAiGwAAFC15k2MAAAA9rIBAACRDQCAyAYAAEQ2AAC8FUclvyU+Pv7UqVNK52vv2bNn9+7dXC531qxZ/v7+rblWtbW1X375ZWlpqYGBwfLly01MTOp+9NNPP129elVfX58Qsnr1agMDg5buDE3TS5YsycvL4/F4P/30U6dOnRTtSUlJq1ev5nA4Y8eOjYiIaOUN/7Zebd68ed++faampoSQ77//3s7OrpU7lpWV9csvv8TFxdW1qLdQb+uVegsllUoXLFhQWFgokUhmzZo1ePBgtdfqbV1Sb6Hkcvk333xTUFDAYrG+++47JycnJgyqt/WqSbWi/2c//vhjSEjInDlz6jcWFxePGDFCIpFUVVWNGDFCLBbTrWjTpk1//PEHTdPHjx//8ccf6/8oMjKyvLy8NTtz5syZefPm0TR9/fr1Dz/8UNEokUgCAwOrqqrEYvHYsWNLS0vp1tVor2ia/uKLL3Jzc2k1Wbdu3YgRIyZMmFDXovZCNdortRdq//79S5cupWn65cuX/v7+TKhVo11Se6Hi4+MXLFhA0/SVK1eY89fXaK+aWCsVTIw0eu+RGzdueHp6cjgcAwMDOzu7vLy81vwQy8zMHDRoECFk0KBBly9frv/59PDhw0WLFkVGRu7fv7/VOuPr60sI6d27d05OjqLx/v37tra2BgYGXC7X09MzPT29lT/nG+0VISQ3N3fNmjVRUVFr165t/Z1ZW1vbVatW1W9Re6Ea7ZXaCxUSEjJ79mzF/hqHw2FCrRrtktoLNXTo0B9++IEQUlhYaGxszJBB1WivmlirZk+MNPHeIwKBwNDQUPH/enp6VVVVLVoCpV6Zm5srZjz09fUFAkFde01NTWxs7NSpU6VS6aRJk1xdXbt27drSm6d+KTgcjlwuZ7FY9Rv19fVbuj5N7BUhJDQ0NDo62sDA4OOPP05JSfHz82vNXgUGBhYWFr6tn2opVKO9UnuhdHV1FcWZPXv2559/zoRaNdoltReKEMJisebNm5eQkPD7778zZ1A17FUTa9XsyB43bty4ceP+62IGBgZ1WVldXW1kZNSi66/Uq08//bS6ulrx1nXbRjGqYmNj+Xw+n88fMGDA7du3WyGyDQwMFJ1R7IAokrGV69PEXhFCJk+erPi08/Pzu3nzZuv/gal3IDWd2gv17NmzTz75JCYmZvjw4QypVcMuMWRELV++vLS0NCIi4sSJEzo6OgwZVEq9amKtWuqMkV69emVmZorF4qqqqvz8/C5durRmLTw8PFJSUgghKSkpffr0qWsvKCiIjIxUTGZlZma6uLi0ZmeuX79e9wnh6Oj48OHDyspKsVicnp7u5ubWysOl0V4JBIIRI0YIhUKaplNTU1unPo0eX6n7f7UXqtFeqb1QJSUl06dP//LLL8eMGcOQWjXaJbUX6vDhw4pJBj6fz2KxFLsmah9UjfaqibXiqLw3mzdvtrW1DQgIiI2NjYqKoml6zpw5PB6vNSsSGRn59ddfR0VF8Xg8xXksdb0KCwuLiIjgcrljxoxxdHRsna/VFy9enDhxomIe6dixY0KhMCIiYv78+dOmTaNpOiIiwtLSsvW/7Dfaqzlz5ii+iHh5eSmOB7Q+iqIIIQwp1Nt6pd5CrVmzprKycvXq1atWraIoavz48Wqv1du6pN5CBQUFzZ8/PyYmRnFCy5kzZ5gwqN7Wq6bUCvcYAQDQGLiUBgAAkQ0AAIhsAABENgAAILIBAACRDQCAyAYAAIb6f0hjCQseq+QoAAAAAElFTkSuQmCC" />

A linear discriminative classifier would attempt to draw a straight line separating the two sets of data, and thereby create a model for classification. For two-dimensional data like that shown here, this is a task we could do by hand. But immediately we see a problem: there is more than one possible dividing line that can perfectly discriminate between the two classes!

We can draw them as follows (Figure 5-54):

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plt.plot([0.6], [2.1], 'x', color='red', markeredgewidth=2, markersize=10)
for m, b in [(1, 0.65), (0.5, 1.6), (-0.2, 2.9)]:
    plt.plot(xfit, m * xfit + b, '-k')
plt.xlim(-1, 3.5);

> **Figure 5-54: _Three perfect linear discriminative classifiers for our data_**<br><img width="300" src="data:image/png;base64, ..." />

These are three very different separators that, nevertheless, perfectly discriminate between these samples. Depending on which you choose, a new data point (e.g., the one marked by the “X” in Figure 5-54) will be assigned a different label! Evidently our simple intuition of “drawing a line between classes” is not enough, and we need to think a bit deeper.

# Support Vector Machines: Maximizing the Margin

Support vector machines offer one way to improve on this. The intuition is this: rather than simply drawing a zero-width line between the classes, we can draw around each line a margin of some width, up to the nearest point. Here is an example of how this might look (Figure 5-55):

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
for m, b, d in [(1, 0.65, 0.33), (0.5, 1.6, 0.55), (-0.2, 2.9, 0.2)]:
    yfit = m * xfit + b
    plt.plot(xfit, yfit, '-k')
    plt.fill_between(xfit, yfit - d, yfit + d, edgecolor='none', color='#AAAAAA',
                     alpha=0.4)
plt.xlim(-1, 3.5);

> **Figure 5-55: _Visualization of “margins” within discriminative classifiers_**<br><img width="300" src="data:image/png;base64, ..." />

In support vector machines, the line that maximizes this margin is the one we will choose as the optimal model. Support vector machines are an example of such a maximum margin estimator.

# Fitting a support vector machine

Let’s see the result of an actual fit to this data: we will use Scikit-Learn’s support vector classifier to train an SVM model on this data. For the time being, we will use a linear kernel and set the C parameter to a very large number (we’ll discuss the meaning of these in more depth momentarily):

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"
model = SVC(kernel='linear', C=1E10)
model.fit(X, y)

Output:<br>`
SVC(C=10000000000.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
        `

To better visualize what’s happening here, let’s create a quick convenience function that will plot SVM decision boundaries for us (Figure 5-56):

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a two-dimensional SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(model);

> **Figure 5-56: _A support vector machine classifier fit to the data, with margins (dashed lines) and support vectors (circles) shown_**<br><img width="300" src="data:image/png;base64, ..." />

This is the dividing line that maximizes the margin between the two sets of points. Notice that a few of the training points just touch the margin; they are indicated by the black circles in Figure 5-56. These points are the pivotal elements of this fit, and are known as the support vectors, and give the algorithm its name. In Scikit-Learn, the identity of these points is stored in the support_vectors_ attribute of the classifier:

In [ ]:
model.support_vectors_

Output:<br>`
array([[ 0.44359863,    3.11530945],
       [ 2.33812285,    3.43116792],
       [ 2.06156753,    1.96918596]])
        `

A key to this classifier’s success is that for the fit, only the position of the support vectors matters; any points further from the margin that are on the correct side do not modify the fit! Technically, this is because these points do not contribute to the loss function used to fit the model, so their position and number do not matter so long as they do not cross the margin.

We can see this, for example, if we plot the model learned from the first 60 points and first 120 points of this dataset (Figure 5-57):

In [ ]:
def plot_svm(N=10, ax=None):
    X, y = make_blobs(n_samples=200, centers=2,
                      random_state=0, cluster_std=0.60)
    X = X[:N]
    y = y[:N]
    model = SVC(kernel='linear', C=1E10)
    model.fit(X, y)
    ax = ax or plt.gca()
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    ax.set_xlim(-1, 4)
    ax.set_ylim(-1, 6)
    plot_svc_decision_function(model, ax)
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)
for axi, N in zip(ax, [60, 120]):
    plot_svm(N, axi)
    axi.set_title('N = {0}'.format(N))

> **Figure 5-57: _The influence of new training points on the SVM model_**<br><img width="300" src="data:image/png;base64, ..." />

In the left panel, we see the model and the support vectors for 60 training points. In the right panel, we have doubled the number of training points, but the model has not changed: the three support vectors from the left panel are still the support vectors from the right panel. This insensitivity to the exact behavior of distant points is one of the strengths of the SVM model.

If you are running this notebook live, you can use IPython’s interactive widgets to view this feature of the SVM model interactively (Figure 5-58):

In [ ]:
from ipywidgets import interact, fixed
interact(plot_svm, N=[10, 200], ax=fixed(None));

> **Figure 5-58: _The first frame of the interactive SVM visualization (see the online appendix for the full version)_**<br><img width="300" src="data:image/png;base64, ..." />

# Beyond linear boundaries: Kernel SVM

Where SVM becomes extremely powerful is when it is combined with kernels. We have seen a version of kernels before, in the basis function regressions of “In Depth: Linear Regression” on page 390. There we projected our data into higher-dimensional space defined by polynomials and Gaussian basis functions, and thereby were able to fit for nonlinear relationships with a linear classifier.

In SVM models, we can use a version of the same idea. To motivate the need for kernels, let’s look at some data that is not linearly separable (Figure 5-59):

In [ ]:
from sklearn.datasets.samples_generator import make_circles
X, y = make_circles(100, factor=.1, noise=.1)
clf = SVC(kernel='linear').fit(X, y)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(clf, plot_support=False);

> **Figure 5-59: _A linear classifier performs poorly for nonlinear boundaries_**<br><img width="300" src="data:image/png;base64, ..." />

It is clear that no linear discrimination will ever be able to separate this data. But we can draw a lesson from the basis function regressions in “In Depth: Linear Regression” on page 390, and think about how we might project the data into a higher dimension such that a linear separator would be sufficient. For example, one simple projection we could use would be to compute a radial basis function centered on the middle clump:

In [ ]:
r = np.exp(-(X ** 2).sum(1))

We can visualize this extra data dimension using a three-dimensional plot - if you are running this notebook live, you will be able to use the sliders to rotate the plot (Figure 5-60):

In [ ]:
from mpl_toolkits import mplot3d
def plot_3D(elev=30, azim=30, X=X, y=y):
    ax = plt.subplot(projection='3d')
    ax.scatter3D(X[:, 0], X[:, 1], r, c=y, s=50, cmap='autumn')
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('r')
interact(plot_3D, elev=[-90, 90], azip=(-180, 180),
         X=fixed(X), y=fixed(y));

> **Figure 5-60: _A third dimension added to the data allows for linear separation_**<br><img width="300" src="data:image/png;base64, ..." />

We can see that with this additional dimension, the data becomes trivially linearly separable, by drawing a separating plane at, say, r=0.7.

Here we had to choose and carefully tune our projection; if we had not centered our radial basis function in the right location, we would not have seen such clean, linearly separable results. In general, the need to make such a choice is a problem: we would like to somehow automatically find the best basis functions to use.

One strategy to this end is to compute a basis function centered at every point in the dataset, and let the SVM algorithm sift through the results. This type of basis function transformation is known as a kernel transformation, as it is based on a similarity relationship (or kernel) between each pair of points.

A potential problem with this strategy - projecting N points into N dimensions - is that it might become very computationally intensive as N grows large. However, because of a neat little procedure known as the kernel trick, a fit on kernel-transformed data can be done implicitly - that is, without ever building the full N-dimensional representation of the kernel projection! This kernel trick is built into the SVM, and is one of the reasons the method is so powerful.

In Scikit-Learn, we can apply kernelized SVM simply by changing our linear kernel to an RBF (radial basis function) kernel, using the kernel model hyperparameter (Figure 5-61):

In [ ]:
clf = SVC(kernel='rbf', C=1E6)
clf.fit(X, y)

Output:<br>`
SVC(C=1000000.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
        `

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(clf)
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
            s=300, lw=1, facecolors='none');

> **Figure 5-61: _Kernel SVM fit to the data_**<br><img width="300" src="data:image/png;base64, ..." />

Using this kernelized support vector machine, we learn a suitable nonlinear decision boundary. This kernel transformation strategy is used often in machine learning to turn fast linear methods into fast nonlinear methods, especially for models in which the kernel trick can be used.

# Tuning the SVM: Softening margins

Our discussion so far has centered on very clean datasets, in which a perfect decision boundary exists. But what if your data has some amount of overlap? For example, you may have data like this (Figure 5-62):

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=1.2)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

> **Figure 5-62: _Data with some level of overlap_**<br><img width="300" src="data:image/png;base64, ..." />

To handle this case, the SVM implementation has a bit of a fudge-factor that “softens” the margin; that is, it allows some of the points to creep into the margin if that allows a better fit. The hardness of the margin is controlled by a tuning parameter, most often known as C. For very large C, the margin is hard, and points cannot lie in it. For smaller C, the margin is softer, and can grow to encompass some points.

The plot shown in Figure 5-63 gives a visual picture of how a changing C parameter affects the final fit, via the softening of the margin:

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=0.8)
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)
for axi, C in zip(ax, [10.0, 0.1]):
    model = SVC(kernel='linear', C=C).fit(X, y)
    axi.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    plot_svc_decision_function(model, axi)
    axi.scatter(model.support_vectors_[:, 0],
                model.support_vectors_[:, 1],
                s=300, lw=1, facecolors='none');
    axi.set_title('C = {0:.1f}'.format(C), size=14)

> **Figure 5-63: _The effect of the C parameter on the support vector fit_**<br><img width="300" src="data:image/png;base64, ..." />

The optimal value of the C parameter will depend on your dataset, and should be tuned via cross-validation or a similar procedure (refer back to “Hyperparameters and Model Validation” on page 359 for further information).

# Example: Face Recognition

As an example of support vector machines in action, let’s take a look at the facial recognition problem. We will use the Labeled Faces in the Wild dataset, which consists of several thousand collated photos of various public figures. A fetcher for the dataset is built into Scikit-Learn:

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

Output:<br>`
['Ariel Sharon' 'Colin Powell' 'Donald Rumsfeld' 'George W Bush'
 'Gerhard Schroeder' 'Hugo Chavez' 'Junichiro Koizumi' 'Tony Blair']
(1348, 62, 47)
    `

Let’s plot a few of these faces to see what we’re working with (Figure 5-64):

In [ ]:
fig, ax = plt.subplots(3, 5)
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=faces.target_names[faces.target[i]])

> **Figure 5-64: _Examples from the Labeled Faces in the Wild dataset_**<br><img width="300" src="data:image/png;base64, ..." />

Each image contains [62×47] or nearly 3,000 pixels. We could proceed by simply using each pixel value as a feature, but often it is more effective to use some sort of preprocessor to extract more meaningful features; here we will use a principal component analysis (see “In Depth: Principal Component Analysis” on page 433) to extract 150 fundamental components to feed into our support vector machine classifier. We can do this most straightforwardly by packaging the preprocessor and the classifier into a single pipeline:

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import RandomizedPCA
from sklearn.pipeline import make_pipeline
pca = RandomizedPCA(n_components=150, whiten=True, random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

For the sake of testing our classifier output, we will split the data into a training and testing set:

In [ ]:
from sklearn.cross_validation import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(faces.data, faces.target,
                                                random_state=42)

Finally, we can use a grid search cross-validation to explore combinations of parameters. Here we will adjust C (which controls the margin hardness) and gamma (which controls the size of the radial basis function kernel), and determine the best model:

In [ ]:
from sklearn.grid_search import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)
%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)

Output:<br>`
CPU times: user 47.8 s, sys: 4.08 s, total: 51.8 s
Wall time: 26 s
{'svc__gamma': 0.001, 'svc__C': 10}
    `

The optimal values fall toward the middle of our grid; if they fell at the edges, we would want to expand the grid to make sure we have found the true optimum.

Now with this cross-validated model, we can predict the labels for the test data, which the model has not yet seen:

In [ ]:
model = grid.best_estimator_
yfit = model.predict(Xtest)

Let’s take a look at a few of the test images along with their predicted values (Figure 5-65):

In [ ]:
fig, ax = plt.subplots(4, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(62, 47), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == ytest[i] else 'red')
fig.suptitle('Predicted Names; Incorrect Labels in Red', size=14);

> **Figure 5-65: _Labels predicted by our model_**<br><img width="300" src="data:image/png;base64, ..." />

Out of this small sample, our optimal estimator mislabeled only a single face (Bush’s face in the bottom row was mislabeled as Blair). We can get a better sense of our estimator’s performance using the classification report, which lists recovery statistics label by label:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, yfit,
                            target_names=faces.target_names))

Output:<br>`
                    precision    recall    f1-score     support
     Ariel Sharon        0.65       0.73       0.69           15
     Colin Powell        0.81       0.87       0.84           68
  Donald Rumsfeld        0.75       0.87       0.81           31
    George W Bush        0.93       0.83       0.88          126
Gerhard Schroeder        0.86       0.78       0.82           23
      Hugo Chavez        0.93       0.70       0.80           20
Junichiro Koizumi        0.80       1.00       0.89           12
       Tony Blair        0.83       0.93       0.88           42
      avg / total        0.85       0.85       0.85          337
    `

We might also display the confusion matrix between these classes (Figure 5-66):

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=faces.target_names,
            yticklabels=faces.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');

> **Figure 5-66: _Confusion matrix for the faces data_**<br><img width="300" src="data:image/png;base64, ..." />

This helps us get a sense of which labels are likely to be confused by the estimator.

For a real-world facial recognition task, in which the photos do not come precropped into nice grids, the only difference in the facial classification scheme is the feature selection: you would need to use a more sophisticated algorithm to find the faces, and extract features that are independent of the pixellation. For this kind of application, one good option is to make use of OpenCV, which among other things, includes pretrained implementations of state-of-the-art feature extraction tools for images in general and faces in particular.

# Support Vector Machine Summary

We have seen here a brief intuitive introduction to the principals behind support vector machines. These methods are a powerful classification method for a number of reasons:

- Their dependence on relatively few support vectors means that they are very compact models, and take up very little memory.
- Once the model is trained, the prediction phase is very fast.
- Because they are affected only by points near the margin, they work well with high-dimensional data—even data with more dimensions than samples, which is a challenging regime for other algorithms.
- Their integration with kernel methods makes them very versatile, able to adapt to many types of data.

However, SVMs have several disadvantages as well:

- The scaling with the number of samples N is $\mathcal{O}[N^3]$ at worst, or $\mathcal{O}[N^2]$ for efficient implementations. For large numbers of training samples, this computational cost can be prohibitive.
- The results are strongly dependent on a suitable choice for the softening parameter C. This must be carefully chosen via cross-validation, which can be expensive as datasets grow in size.
- The results do not have a direct probabilistic interpretation. This can be estimated via an internal cross-validation (see the probability parameter of SVC), but this extra estimation is costly.

With those traits in mind, I generally only turn to SVMs once other simpler, faster, and less tuning-intensive methods have been shown to be insufficient for my needs. Nevertheless, if you have the CPU cycles to commit to training and cross-validating an SVM on your data, the method can lead to excellent results.